# DIOMIRA

## A nb to develop the cython version of DIOMIRA

In [1]:
from __future__ import print_function
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%env IPYTHONDIR=/Users/jjgomezcadenas/Documents/Development/IPYTHON

env: IPYTHONDIR=/Users/jjgomezcadenas/Documents/Development/IPYTHON


In [3]:
%load_ext Cython

In [4]:
import pandas as pd
import numpy as np
import FEParam as FP
import tables
from time import time
from Util import *

### Classical fib function

In [ ]:
def fib(n):
    a,b = 0.0, 1.0
    for i in range(n):
        a,b = a+b,a
    return a

In [ ]:
%%cython
def cfib(int n):
    cdef int i
    cdef double a = 0.0, b= 1.0
    for i in range(n):
        a,b = a+b, a
    return a

In [ ]:
%timeit fib(0)

In [ ]:
%timeit fib(90)

In [ ]:
%timeit cfib(0)

In [ ]:
%timeit cfib(90)

In [ ]:
f = 118./6050.

In [ ]:
1./f

### Rebin array

#### Python version

In [ ]:
def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

In [ ]:
cx = np.arange(0.,100., 1)

In [ ]:
%time y = rebin_pmt_array(cx,10)

In [ ]:
%timeit y = rebin_pmt_array(cx,10)

#### Cython version (pure compilation)

In [ ]:
%%cython
import numpy as np
def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

In [ ]:
%timeit rebin_pmt_array(cx,10)

#### cython version, declares variables but still uses np.sum()

In [ ]:
%%cython
import numpy as np
cimport numpy as np
def crebin_pmt_array(double [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j
        double [:] b
    
    lena =   a.shape[0]   
    lenb = a.shape[0]/stride
    
    b = np.zeros(lenb, dtype=np.double)
    
    j=0
    
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    
    
    return np.asarray(b)

In [ ]:
%timeit z = crebin_pmt_array(cx,10)

*NB: the code runs 3 times SLOWER than the cython version*

#### cythonizes the sum

In [ ]:
%%cython
import numpy as np
cimport numpy as np
def c2rebin_pmt_array(double [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j,k
        double [:] b
        double part_sum
    
    lena =   a.shape[0]   
    lenb = a.shape[0]/stride
    
    b = np.zeros(lenb)
    
    j=0
    
    for i in range(lenb):
        part_sum = 0.
        for k in range(j, j + stride): 
            part_sum += a[k] 
        b[i] = part_sum
        j+= stride
    
    return np.asarray(b)

In [ ]:
%timeit z = c2rebin_pmt_array(cx,10)

*IMPROVEMENT*

#### Direct use of nd.array() in function

In [ ]:
%%cython
import numpy as np
cimport numpy as np
def c3rebin_pmt_array(double [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j,k
        double part_sum
      
    lenb = a.shape[0]/stride
    cdef np.ndarray[np.float64_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    for i in range(lenb):
        part_sum = 0.
        for k in range(j, j + stride): 
            part_sum += a[k] 
        b[i] = part_sum
        j+= stride
    
    return b

In [ ]:
%timeit z = c3rebin_pmt_array(cx,10)

*Further improvement!*

#### switch off checks

In [ ]:
%%cython
import numpy as np
cimport numpy as np
from cython cimport boundscheck, wraparound
def c4rebin_pmt_array(np.ndarray[np.float64_t, ndim=1] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    cdef:
        int lenb, i, j,k
        double part_sum
      
    lenb = a.shape[0]/stride
    cdef np.ndarray[np.float64_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0.
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
    
    return b

In [ ]:
%timeit z = c4rebin_pmt_array(cx,10)

In [ ]:
%timeit z = c4rebin_pmt_array(cx,10)

In [ ]:
%timeit z = rebin_pmt_array(cx,10)

*About a factor 15 improvement*

### Profiling

In [9]:
import cProfile

In [ ]:
def main():
    cx = np.arange(0.,1000000., 1)
    z = rebin_pmt_array(cx,10)

In [ ]:
cProfile.run('main()', sort='time')

In [ ]:
def cmain():
    cx = np.arange(0.,1000000., 1)
    z = c4rebin_pmt_array(cx,10)

In [ ]:
cProfile.run('cmain()', sort='time')

*NB It seems that the improvement for large arrays can be much larger***

## DIOMIRA

In [ ]:
"""
DIOMIRA
JJGC August 2016

What DIOMIRA does:
1) Reads a MCRD file containing MC waveforms for the 12 PMTs of the EP.
   Each waveform contains number of PEs in bins of 1 ns.
2) Convolves the PE waveform with the response of the FEE electronics.
3) Decimates the waveform, simulating the effect of the DAQ sampling (25 ns bins)
4) Writes a RWF file with the new data and adds the FEE simulation parameters as metadata
"""

from __future__ import print_function
from Util import *
from LogConfig import *
from Configure import configure
from Nh5 import *
from cities import diomira
from SensorsResponse import *

import tables


"""
ChangeLog:

26.9 

Changed types of PMTRWF, SIPMRWF and PMTTWF to Float32 for 
    (compatibility with ART/GATE)

Do not store EPMT and ESIPM (can be computed on the fly)

Change sign of pmtrwf to negative (as produced by the DAQ)
"""

def DIOMIRA(argv):
    DEBUG_LEVEL, INFO, CFP = configure(argv[0],argv[1:])
   
    if INFO:
        print(diomira)

    #wait()
    
    print("""
        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in the PMTs MCRD, 
        and produces both RWF and TWF:
        see: http://localhost:8931/notebooks/Nh5-Event-Model.ipynb#Reconstructed-Objects
        
            
        3. Simulates the response of the tracking plane in the SiPMs MCRD and outputs
            SiPM RWF (not yet implemented, for the time being simply copy the MCRD)

        4. Add a table describing the FEE parameters used for simulation

        5. Copies the tables on geometry, detector data and MC


        """)
    FP.print_FEE()
    #wait()

    PATH_IN =CFP['PATH_IN']
    PATH_OUT =CFP['PATH_OUT']
    FILE_IN =CFP['FILE_IN']
    FILE_OUT =CFP['FILE_OUT']
    FIRST_EVT =CFP['FIRST_EVT']
    LAST_EVT =CFP['LAST_EVT']
    RUN_ALL =CFP['RUN_ALL']
    CLIB =CFP['CLIB']
    CLEVEL =CFP['CLEVEL']
    NEVENTS = LAST_EVT - FIRST_EVT

    print('Debug level = {}'.format(DEBUG_LEVEL))

    logger.info("input path ={}; output path = {}; file_in ={} file_out ={}".format(
        PATH_IN,PATH_OUT,FILE_IN, FILE_OUT))

    logger.info("first event = {} last event = {} nof events requested = {} ".format(
        FIRST_EVT,LAST_EVT,NEVENTS))

    logger.info("Compression library = {} Compression level = {} ".format(
        CLIB,CLEVEL))

    # open the input file 
    with tables.open_file("{}/{}".format(PATH_IN,FILE_IN), "r+") as h5in: 
        # access the PMT raw data in file 

        pmtrd_ = h5in.root.pmtrd
        sipmrd_ = h5in.root.sipmrd

        #pmtrd_.shape = (nof_events, nof_sensors, wf_length)
        NPMT = pmtrd_.shape[1]
        NSIPM = sipmrd_.shape[1]
        PMTWL = pmtrd_.shape[2] 
        PMTWL_FEE = int((PMTWL+1)/FP.time_DAQ)
        SIPMWL = sipmrd_.shape[2]
        NEVENTS_DST = pmtrd_.shape[0]

        logger.info("nof PMTs = {} nof  SiPMs = {} nof events in input DST = {} ".format(
        NPMT,NSIPM,NEVENTS_DST))

        logger.info("lof SiPM WF = {} lof PMT WF (MC) = {} lof PMT WF (FEE) = {}".format(
        PMTWL,SIPMWL,PMTWL_FEE))

        #wait()

        #access the geometry and the sensors metadata info

        geom_t = h5in.root.Detector.DetectorGeometry
        pmt_t = h5in.root.Sensors.DataPMT
        sipm_t = h5in.root.Sensors.DataSiPM
        mctrk_t = h5in.root.MC.MCTracks

        
        # open the output file 
        with tables.open_file("{}/{}".format(PATH_OUT,FILE_OUT), "w",
            filters=tables.Filters(complib=CLIB, complevel=CLEVEL)) as h5out:
 
            # create a group to store MC data
            mcgroup = h5out.create_group(h5out.root, "MC")
            # copy the mctrk table
            mctrk_t.copy(newparent=mcgroup)

            # create a group  to store geom data
            detgroup = h5out.create_group(h5out.root, "Detector")
            # copy the geom table
            geom_t.copy(newparent=detgroup)

            # create a group  store sensor data
            sgroup = h5out.create_group(h5out.root, "Sensors")
            # copy the pmt table
            pmt_t.copy(newparent=sgroup)
            # copy the sipm table
            sipm_t.copy(newparent=sgroup)

            # create a table to store Energy plane FEE data and hang it from MC group
            fee_table = h5out.create_table(mcgroup, "FEE", FEE,
                          "EP-FEE parameters",
                           tables.Filters(0))

            # fill table
            FEE_param_table(fee_table)

            # create a group to store RawData
            rgroup = h5out.create_group(h5out.root, "RD")
            
            # create an extensible array to store the RWF waveforms
            pmtrwf = h5out.create_earray(h5out.root.RD, "pmtrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            
            # create an extensible array to store the TWF waveforms
            pmttwf = h5out.create_earray(h5out.root.RD, "pmttwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            

            sipmrwf = h5out.create_earray(h5out.root.RD, "sipmrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NSIPM, SIPMWL), 
                                    expectedrows=NEVENTS_DST)

            # #create an extensible array to store the energy in PES of PMTs 
            # epmt = h5out.create_earray(h5out.root.RD, "epmt", 
            #                         atom=tables.FloatAtom(), 
            #                         shape=(0, NPMT), 
            #                         expectedrows=NEVENTS_DST)

            # # create an extensible array to store the energy in PES of SiPMs 
            # esipm = h5out.create_earray(h5out.root.RD, "esipm", 
            #                         atom=tables.FloatAtom(), 
            #                         shape=(0, NSIPM), 
            #                         expectedrows=NEVENTS_DST)

            
            if NEVENTS > NEVENTS_DST and RUN_ALL == False:
                print("""
                Refusing to run: you have requested
                FIRST_EVT = {}
                LAST_EVT  = {}
                Thus you want to run over {} events
                but the size of the DST is {} events.
                Please change your choice or select RUN_ALL = TRUE
                to run over the whole DST when this happens
                """.format(FIRST_EVT,LAST_EVT,NEVENTS,NEVENTS_DST))
                sys.exit(0)
            elif  NEVENTS > NEVENTS_DST and RUN_ALL == True:
                FIRST_EVT = 0
                LAST_EVT = NEVENTS_DST
                NEVENTS = NEVENTS_DST


            for i in range(FIRST_EVT,LAST_EVT):
                logger.info("-->event number ={}".format(i))

                #simulate PMT response and return an array with RWF
                dataPMT = simulate_pmt_response(i,pmtrd_)

                #convert to float
                dataPMT.astype(float) 
                #TWF
                 
                truePMT = rebin_signal(i,pmtrd_, int(FP.time_DAQ))
                truePMT.astype(float)
                
                logger.info("truePMT shape ={}".format(truePMT.shape))
                logger.info("dataPMT shape ={}".format(dataPMT.shape))
                
                #RWF for pmts
                pmtrwf.append(dataPMT.reshape(1, NPMT, PMTWL_FEE))
                #pmtrd.append(dataPMT.reshape(1, NPMT, PMTWL))
                
                #TWF for pmts
                pmttwf.append(truePMT.reshape(1, NPMT, PMTWL_FEE))
                #pmtrd.append(dataPMT.reshape(1, NPMT, PMTWL))
                   
                #simulate SiPM response and return an array with new WF
                dataSiPM = simulate_sipm_response(i,sipmrd_)
                dataSiPM.astype(float)
                
                #append to SiPM EARRAY
                sipmrwf.append(dataSiPM.reshape(1, NSIPM, SIPMWL))

                # #fill ene_pmt vector
                # enePMT = energy_pes(i, pmtrd_)
                # #append to epmt EARRAY
                # epmt.append(enePMT.reshape(1, NPMT))

                # #fill ene_sipm vector
                # eneSIPM = energy_pes(i, sipmrd_)
                # esipm.append(eneSIPM.reshape(1, NSIPM))

            pmtrwf.flush()
            pmttwf.flush()
            sipmrwf.flush()
            #epmt.flush()
            #esipm.flush()


    print("Leaving Diomira. Safe travels!")



  

In [ ]:
%less ../../Config/DIOMIRA_NA_ZLIB_test2.csv

### Profile DIOMIRA

In [ ]:
cProfile.run("DIOMIRA(['DIOMIRA','-i','-d','INFO','-c','../../Config/DIOMIRA_NA_ZLIB_test2.csv'])", sort='time')

*Profiling shows that rebin_array is one of the main issues. Start cythonizing it

In [25]:
%%cython
from __future__ import print_function
from Util import *
from LogConfig import *
import FEParam as FP
import SPE as SP
import FEE2 as FE

#import tables
from FEE2 import down_scale_signal_

import numpy as np
cimport numpy as np
from cython cimport boundscheck, wraparound

def rebin_pmt_array(np.ndarray[np.int32_t, ndim=1] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    logger.info("-->*cython::rebin_pmt_array")
    cdef:
        int lenb, i, j,k
        int part_sum
      
    lenb = (a.shape[0]+1)/stride
    cdef np.ndarray[np.int32_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
    
    return b


"""
Code
"""

def FEE_param_table(fee_table):
    """
    Stores the parameters of the EP FEE simulation 
    """
    
    print("calling the cython version")
    
    row = fee_table.row
    row['offset'] = FP.offset
    row['pmt_gain'] = FP.PMT_GAIN
    row['V_gain'] = FP.V_GAIN
    row['R'] = FP.R
    row['C12'] = FP.C12
    row['CO12'] = FP.C12 # to be rewritten by ISIDORA
    row['time_step'] = FP.time_step
    row['time_daq'] = FP.time_DAQ
    row['freq_LPF'] = FP.freq_LPF
    row['freq_HPF'] = 1./(2*pi*FP.R*FP.C)
    row['LSB'] = FP.LSB
    row['volts_to_adc'] = FP.voltsToAdc/volt
    row['noise_fee_rms'] = FP.NOISE_FEE
    row['noise_adc'] = FP.NOISE_ADC
    
    row.append()
    

def energy_pes(event_number, sensord):
    """
    Sum the WFs of PMTs and SiPMs (MC) and store the total energy in PES
    """     
    rdata = []

    for j in range(sensord.shape[1]):
        swf = sensord[event_number, j]
        ene = np.sum(swf)
        rdata.append(ene)
        
    return np.array(rdata) 

def simulate_sipm_response(event_number,sipmrd_):
    """
    For the moment use a dummy rutne that simply copies the sipm EARRAY
    """
    rdata = []

    for j in range(sipmrd_.shape[1]):
        rdata.append(sipmrd_[event_number, j])
    return np.array(rdata)


def simulate_pmt_response(event_number,pmtrd_):
    """
    Sensor Response
    Given a signal in PE (photoelectrons in bins of 1 ns) and the response function of 
    for a single photoelectron (spe) and front-end electronics (fee)
    this function produces the PMT raw data (adc counts bins 25 ns)

    pmtrd_ dataset that holds the PMT PE data for each PMT
    pmtrd25 dataset to be created with adc counts, bins 25 ns 
    after convoluting with electronics
    """
    
    logger.info("-->*cython::simulate_pmt_response, event ={}".format(event_number))
  
    rdata = []

    for j in range(pmtrd_.shape[1]):
        logger.info("-->PMT number ={}".format(j))
                
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        
        fee = FE.FEE(C=FP.C12[j],R= FP.R, f=FP.freq_LPF, RG=FP.V_GAIN) 
        spe = SP.SPE(pmt_gain=FP.PMT_GAIN,x_slope = 5*ns,x_flat = 1*ns)
    
        signal_PMT = spe.SpePulseFromVectorPE(pmt) #PMT response

        #Front end response to PMT pulse (in volts)
        signal_fee = fee.FEESignal(signal_PMT, noise_rms=FP.NOISE_FEE) 

        #Signal out of DAQ
        #positive signal convention
        #signal_daq = fee.daqSignal(signal_fee, noise_rms=0) - FP.offset
        #negative signals convention!

        signal_daq = FP.offset -fee.daqSignal(signal_fee, noise_rms=0) 
    
        rdata.append(signal_daq)
    return np.array(rdata)

def decimate_signal(event_number,pmtrd_):
    """
    Decimates the MCRD signal to produce TWF (pes, bins 25 ns)
    """
  
    rdata = []

    for j in range(pmtrd_.shape[1]):
        logger.debug("-->PMT number ={}".format(j))
                
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = down_scale_signal_(pmt, int(FP.time_DAQ))
        
        
        rdata.append(twf)
    return np.array(rdata)

# def rebin_pmt_array(a,stride):
#     """
#     rebins pmt array a according to stride
#     there is a rebin_array in util which uses
#     lenb = (len(a))/int(stride)
#     this version uses (lean(a)+1) to correct from the fact that the
#     MCRD is 599999 channels (should be 600000)
#     """
    
#     lenb = (len(a)+1)/int(stride)
#     b = np.zeros(lenb)
#     j=0
#     for i in range(lenb):
#         b[i] = np.sum(a[j:j+stride])
#         j+= stride
#     return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    rdata = []
    logger.info("-->*cython::rebin_signal, event ={}".format(event_number))

    for j in range(pmtrd_.shape[1]):
        logger.info("-->PMT number ={}".format(j))
                
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

  


In [6]:
rebin_signal

<function _cython_magic_82a0b0ceee12bcc9415fcb00039019cf.rebin_signal>

In [26]:
"""
DIOMIRA
JJGC August 2016

What DIOMIRA does:
1) Reads a MCRD file containing MC waveforms for the 12 PMTs of the EP.
   Each waveform contains number of PEs in bins of 1 ns.
2) Convolves the PE waveform with the response of the FEE electronics.
3) Decimates the waveform, simulating the effect of the DAQ sampling (25 ns bins)
4) Writes a RWF file with the new data and adds the FEE simulation parameters as metadata
"""

from __future__ import print_function
from Util import *
from LogConfig import *
from Configure import configure
from Nh5 import *
from cities import diomira
#from SensorsResponse import *

import tables


"""
ChangeLog:

26.9 

Changed types of PMTRWF, SIPMRWF and PMTTWF to Float32 for 
    (compatibility with ART/GATE)

Do not store EPMT and ESIPM (can be computed on the fly)

Change sign of pmtrwf to negative (as produced by the DAQ)
"""

def DIOMIRA(argv):
    DEBUG_LEVEL, INFO, CFP = configure(argv[0],argv[1:])
   
    if INFO:
        print(diomira)

    #wait()
    
    print("""
        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in the PMTs MCRD, 
        and produces both RWF and TWF:
        see: http://localhost:8931/notebooks/Nh5-Event-Model.ipynb#Reconstructed-Objects
        
            
        3. Simulates the response of the tracking plane in the SiPMs MCRD and outputs
            SiPM RWF (not yet implemented, for the time being simply copy the MCRD)

        4. Add a table describing the FEE parameters used for simulation

        5. Copies the tables on geometry, detector data and MC


        """)
    FP.print_FEE()
    #wait()

    PATH_IN =CFP['PATH_IN']
    PATH_OUT =CFP['PATH_OUT']
    FILE_IN =CFP['FILE_IN']
    FILE_OUT =CFP['FILE_OUT']
    FIRST_EVT =CFP['FIRST_EVT']
    LAST_EVT =CFP['LAST_EVT']
    RUN_ALL =CFP['RUN_ALL']
    CLIB =CFP['CLIB']
    CLEVEL =CFP['CLEVEL']
    NEVENTS = LAST_EVT - FIRST_EVT

    print('Debug level = {}'.format(DEBUG_LEVEL))

    logger.info("input path ={}; output path = {}; file_in ={} file_out ={}".format(
        PATH_IN,PATH_OUT,FILE_IN, FILE_OUT))

    logger.info("first event = {} last event = {} nof events requested = {} ".format(
        FIRST_EVT,LAST_EVT,NEVENTS))

    logger.info("Compression library = {} Compression level = {} ".format(
        CLIB,CLEVEL))

    # open the input file 
    with tables.open_file("{}/{}".format(PATH_IN,FILE_IN), "r+") as h5in: 
        # access the PMT raw data in file 

        pmtrd_ = h5in.root.pmtrd
        sipmrd_ = h5in.root.sipmrd

        #pmtrd_.shape = (nof_events, nof_sensors, wf_length)
        NPMT = pmtrd_.shape[1]
        NSIPM = sipmrd_.shape[1]
        PMTWL = pmtrd_.shape[2] 
        PMTWL_FEE = int((PMTWL+1)/FP.time_DAQ)
        SIPMWL = sipmrd_.shape[2]
        NEVENTS_DST = pmtrd_.shape[0]

        logger.info("nof PMTs = {} nof  SiPMs = {} nof events in input DST = {} ".format(
        NPMT,NSIPM,NEVENTS_DST))

        logger.info("lof SiPM WF = {} lof PMT WF (MC) = {} lof PMT WF (FEE) = {}".format(
        PMTWL,SIPMWL,PMTWL_FEE))

        #wait()

        #access the geometry and the sensors metadata info

        geom_t = h5in.root.Detector.DetectorGeometry
        pmt_t = h5in.root.Sensors.DataPMT
        sipm_t = h5in.root.Sensors.DataSiPM
        mctrk_t = h5in.root.MC.MCTracks

        
        # open the output file 
        with tables.open_file("{}/{}".format(PATH_OUT,FILE_OUT), "w",
            filters=tables.Filters(complib=CLIB, complevel=CLEVEL)) as h5out:
 
            # create a group to store MC data
            mcgroup = h5out.create_group(h5out.root, "MC")
            # copy the mctrk table
            mctrk_t.copy(newparent=mcgroup)

            # create a group  to store geom data
            detgroup = h5out.create_group(h5out.root, "Detector")
            # copy the geom table
            geom_t.copy(newparent=detgroup)

            # create a group  store sensor data
            sgroup = h5out.create_group(h5out.root, "Sensors")
            # copy the pmt table
            pmt_t.copy(newparent=sgroup)
            # copy the sipm table
            sipm_t.copy(newparent=sgroup)

            # create a table to store Energy plane FEE data and hang it from MC group
            fee_table = h5out.create_table(mcgroup, "FEE", FEE,
                          "EP-FEE parameters",
                           tables.Filters(0))

            # fill table
            FEE_param_table(fee_table)

            # create a group to store RawData
            rgroup = h5out.create_group(h5out.root, "RD")
            
            # create an extensible array to store the RWF waveforms
            pmtrwf = h5out.create_earray(h5out.root.RD, "pmtrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            
            # create an extensible array to store the TWF waveforms
            pmttwf = h5out.create_earray(h5out.root.RD, "pmttwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NPMT, PMTWL_FEE), 
                                    expectedrows=NEVENTS_DST)
            

            sipmrwf = h5out.create_earray(h5out.root.RD, "sipmrwf", 
                                    atom=tables.Float32Atom(), 
                                    shape=(0, NSIPM, SIPMWL), 
                                    expectedrows=NEVENTS_DST)

            # #create an extensible array to store the energy in PES of PMTs 
            # epmt = h5out.create_earray(h5out.root.RD, "epmt", 
            #                         atom=tables.FloatAtom(), 
            #                         shape=(0, NPMT), 
            #                         expectedrows=NEVENTS_DST)

            # # create an extensible array to store the energy in PES of SiPMs 
            # esipm = h5out.create_earray(h5out.root.RD, "esipm", 
            #                         atom=tables.FloatAtom(), 
            #                         shape=(0, NSIPM), 
            #                         expectedrows=NEVENTS_DST)

            
            if NEVENTS > NEVENTS_DST and RUN_ALL == False:
                print("""
                Refusing to run: you have requested
                FIRST_EVT = {}
                LAST_EVT  = {}
                Thus you want to run over {} events
                but the size of the DST is {} events.
                Please change your choice or select RUN_ALL = TRUE
                to run over the whole DST when this happens
                """.format(FIRST_EVT,LAST_EVT,NEVENTS,NEVENTS_DST))
                sys.exit(0)
            elif  NEVENTS > NEVENTS_DST and RUN_ALL == True:
                FIRST_EVT = 0
                LAST_EVT = NEVENTS_DST
                NEVENTS = NEVENTS_DST


            for i in range(FIRST_EVT,LAST_EVT):
                logger.info("-->event number ={}".format(i))
                logger.info("--> simulate_pmt_response: i ={}".format(i))

                #simulate PMT response and return an array with RWF
                dataPMT = simulate_pmt_response(i,pmtrd_)

                #convert to float
                dataPMT.astype(float) 
                #TWF
                
                logger.info("--> rebin_signal: i ={}".format(i))
                truePMT = rebin_signal(i,pmtrd_, int(FP.time_DAQ))
                truePMT.astype(float)
                
                logger.info("truePMT shape ={}".format(truePMT.shape))
                logger.info("dataPMT shape ={}".format(dataPMT.shape))
                
                #RWF for pmts
                pmtrwf.append(dataPMT.reshape(1, NPMT, PMTWL_FEE))
                #pmtrd.append(dataPMT.reshape(1, NPMT, PMTWL))
                
                #TWF for pmts
                pmttwf.append(truePMT.reshape(1, NPMT, PMTWL_FEE))
                #pmtrd.append(dataPMT.reshape(1, NPMT, PMTWL))
                   
                #simulate SiPM response and return an array with new WF
                dataSiPM = simulate_sipm_response(i,sipmrd_)
                dataSiPM.astype(float)
                
                #append to SiPM EARRAY
                sipmrwf.append(dataSiPM.reshape(1, NSIPM, SIPMWL))

                # #fill ene_pmt vector
                # enePMT = energy_pes(i, pmtrd_)
                # #append to epmt EARRAY
                # epmt.append(enePMT.reshape(1, NPMT))

                # #fill ene_sipm vector
                # eneSIPM = energy_pes(i, sipmrd_)
                # esipm.append(eneSIPM.reshape(1, NSIPM))

            pmtrwf.flush()
            pmttwf.flush()
            sipmrwf.flush()
            #epmt.flush()
            #esipm.flush()


    print("Leaving Diomira. Safe travels!")




In [27]:
cProfile.run("DIOMIRA(['DIOMIRA','-i','-d','INFO','-c','../../Config/DIOMIRA_NA_ZLIB_test2.csv'])", sort='time')

INFO:root:Configuration Parameters (CFP) dictionary  = {'FIRST_EVT': 0, 'LAST_EVT': 3, 'FILE_OUT': 'WF_Na_ZLIB_test3_RWF.h5', 'PATH_OUT': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/', ' END ': 1, 'CLIB': 'zlib', 'RUN_ALL': 1, 'PATH_IN': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/', 'CLEVEL': 1, 'FILE_IN': 'WF_Na_1Kevts_comp1_chunk32k.h5'}


Configuration Parameters (CFP) dictionary  = {'FIRST_EVT': 0, 'LAST_EVT': 3, 'FILE_OUT': 'WF_Na_ZLIB_test3_RWF.h5', 'PATH_OUT': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/', ' END ': 1, 'CLIB': 'zlib', 'RUN_ALL': 1, 'PATH_IN': '/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/', 'CLEVEL': 1, 'FILE_IN': 'WF_Na_1Kevts_comp1_chunk32k.h5'}



Leaving there and proceeding for three days toward the east, you reach Diomira, 
a city with sixty silver domes, bronze statues of all the gods, streets paved with lead, 
a crystal theater, a golden cock that crows every morning on a tower. 
All these beauties will already be familiar to the visitor, 
who has seen them also in other cities. 
But the special quality of this city for the man who arrives there on a September evening, 
when the days are growing shorter 
and the multicolored lamps are lighted all at once at the doors of the food stalls 
and from a terrace a woman's voice cries ooh!, is that he feels envy 
toward those who now believe they have once before lived an 
evening identical to this and who think they were happy, that time.

        DIOMIRA:
         1. Reads an MCRD file produced by art/centella, which stores MCRD 
         waveforms for PMTs (bins of 1 ns)
        and the SiPMs (bins of 1 mus)
            

        2. Simulates the response of the energy plane in

input path =/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/WF-NA-ZLIB/; output path = /Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/; file_in =WF_Na_1Kevts_comp1_chunk32k.h5 file_out =WF_Na_ZLIB_test3_RWF.h5


INFO:root:first event = 0 last event = 3 nof events requested = 3 


...

-->PMT number =0


INFO:root:-->*cython::rebin_pmt_array


-->*cython::rebin_pmt_array


         45023 function calls (44994 primitive calls) in 3.261 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7    1.026    0.147    1.026    0.147 {method '_fill_col' of 'tables.tableextension.Row' objects}
       12    0.613    0.051    0.613    0.051 {method 'normal' of 'mtrand.RandomState' objects}
      156    0.573    0.004    0.573    0.004 {numpy.core.multiarray.correlate}
        8    0.340    0.042    0.340    0.042 {method '_append_records' of 'tables.tableextension.Table' objects}
       24    0.130    0.005    0.130    0.005 {scipy.signal.sigtools._linear_filter}
        1    0.108    0.108    1.672    1.672 {_cython_magic_b3792d1d4fc96b0de24710bf906a8a85.simulate_pmt_response}
       13    0.070    0.005    0.070    0.005 {method '_g_read_slice' of 'tables.hdf5extension.Array' objects}
       12    0.064    0.005    0.808    0.067 FEE2.py:156(FEESignal)
        2    0.064    0.032    0.064    0.032 {m

ValueError: Buffer dtype mismatch, expected 'int32_t' but got 'double'

In [ ]:
%%cython
import numpy as np
cimport numpy as np
from cython cimport boundscheck, wraparound
def rebin_pmt_array(np.ndarray[np.float64_t, ndim=1] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    print("*cyhon version*")
    cdef:
        int lenb, i, j,k
        double part_sum
      
    lenb = (a.shape[0]+1)/stride
    cdef np.ndarray[np.float64_t, ndim=1] b = np.zeros(lenb) 
    
    j=0
    with boundscheck(False), wraparound(False):
        for i in range(lenb):
            part_sum = 0.
            for k in range(j, j + stride): 
                part_sum += a[k] 
                b[i] = part_sum
            j+= stride
    
    return b

In [ ]:
rebin_pmt_array

In [ ]:
cProfile.run("DIOMIRA(['DIOMIRA','-i','-d','INFO','-c','../../Config/DIOMIRA_NA_ZLIB_test2.csv'])", sort='time')

In [ ]:
h5f =tables.open_file('/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/WF_Na_ZLIB_test100_RWF.h5')

In [ ]:
pmtrwf = h5f.root.RD.pmtrwf

### Python version

In [ ]:

def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    #t1 = time()
    #print("rebin_pmt_array")
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    #t2 = time()
    #print('time = {}'.format(t2-t1))
    return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    t1 = time()
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        #print("pmt = {}: Call rebin_pmt_array".format(j))
        t2 = time()
        twf = rebin_pmt_array(pmt, stride)
        #print('time = {}'.format(t2-t1))
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    t1 = time()
    for event in list_of_events:
        print("event = {}. Call rebin signal".format(event))
        rebin_signal(event,pmtrd, stride)
        t2 = time()
        print('time = {}'.format(t2-t1))
        
        

In [ ]:
rebin_signals

In [ ]:
%time rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

### Cython version (just compile python)

In [ ]:
%%cython
import pandas as pd
import numpy as np
import FEParam as FP
import tables

def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    for event in list_of_events:
        rebin_signal(event,pmtrd, stride)

In [ ]:
rebin_signals

In [ ]:
def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    t1 = time()
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        #print("pmt = {}: Call rebin_pmt_array".format(j))
        t2 = time()
        twf = rebin_pmt_array(pmt, stride)
        #print('time = {}'.format(t2-t1))
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    t1 = time()
    for event in list_of_events:
        print("event = {}. Call rebin signal".format(event))
        rebin_signal(event,pmtrd, stride)
        t2 = time()
        print('time = {}'.format(t2-t1))
        
        

In [ ]:
rebin_signals

In [ ]:
rebin_signal

In [ ]:
rebin_pmt_array

In [ ]:
3000.*25./1000.

In [ ]:
%time rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

### Cython version (supposedly optimized)

In [ ]:
%%cython
import pandas as pd
import numpy as np
import FEParam as FP
import tables
from time import time

def rebin_pmt_array(float [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    cdef int lenb
    lenb = (len(a)+1)/stride
    b = np.zeros(lenb, dtype = np.float)
    cdef int j=0
    cdef int i
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

# def rebin_signal(int event_number, pmtrd_, int stride):
#     """
#     rebins the MCRD signal to produce TWF (pes, bins 25 ns)
#     """
    
#     #rdata = np.zeros(pmtrd_.shape[1], dtype = np.object)
#     rdata = []
#     cdef int j
#     for j in range(pmtrd_.shape[1]):
#         pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
#         twf = rebin_pmt_array(pmt, stride)
        
#         rdata.append(twf)
#     return np.array(rdata)

# def rebin_signals(pmtrd, int stride, list_of_events):
#     """
#     rebin the signals in a list of events
#     """
#     cdef int event
#     for event in list_of_events:
#         rebin_signal(event,pmtrd, stride)

In [ ]:
rebin_pmt_array

In [ ]:
%timeit rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

### Cython does not improve over python code in this case!

In [ ]:
h5f.close()